In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import gc

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from keras import optimizers
from keras.applications import Xception

Using TensorFlow backend.


In [14]:
#declare the training and testing images, labels
train_images=[]
test_images=[]
train_labels=[]
test_labels=[]

#number of classes
num_classes=10

In [15]:
training_dir="10-monkey-species/training"
validation_dir="10-monkey-species/validation"

In [16]:
#Reading training data
total_images = 0
total_count = 0
image_size = 299
label = -1
training = os.listdir(training_dir)

for file in training:
    file_name = os.listdir(os.path.join(training_dir, file))
    if len(file_name)>0:
        label += 1
        print(file + " monkeys = " + str(len(file_name)))
        total_images += len(file_name)
        for image in file_name:
            path = os.path.join(training_dir, file, image)
            img = cv2.imread(path)
            img = cv2.resize(img, (image_size, image_size))
            img = np.array(img)
            train_images.append(img)
            train_labels.append(label)
            total_count += 1

len(train_labels)

n0 monkeys = 105
n1 monkeys = 111
n2 monkeys = 110
n3 monkeys = 122
n4 monkeys = 105
n5 monkeys = 113
n6 monkeys = 106
n7 monkeys = 114
n8 monkeys = 106
n9 monkeys = 106


1098

In [17]:
#Reading test data
total_images = 0
total_count = 0
image_size = 299
label = -1
test = os.listdir(validation_dir)

for file in test:
    file_name = os.listdir(os.path.join(validation_dir, file))
    if len(file_name)>0:
        label += 1
        print(file + " monkeys = " + str(len(file_name)))
        total_images += len(file_name)
        for image in file_name:
            path = os.path.join(validation_dir, file, image)
            img = cv2.imread(path)
            img = cv2.resize(img, (image_size, image_size))
            img = np.array(img)
            test_images.append(img)
            test_labels.append(label)
            total_count += 1

len(test_labels)

n0 monkeys = 26
n1 monkeys = 28
n2 monkeys = 27
n3 monkeys = 30
n4 monkeys = 26
n5 monkeys = 28
n6 monkeys = 26
n7 monkeys = 28
n8 monkeys = 27
n9 monkeys = 26


272

In [18]:
random.seed(1)
idx = list(range(len(train_images)))
shuffled = random.shuffle(idx)
new_train = [train_images[i] for i in idx]
new_train_labels = [train_labels[i] for i in idx]

In [19]:
#normalizing the data
new_train = np.array(new_train)
test_images = np.array(test_images)
new_train = new_train.astype('float32')/255
test_images = test_images.astype('float32')/255

new_train_labels = np.array(new_train_labels)
test_labels = np.array(test_labels)
new_train_labels = keras.utils.to_categorical(new_train_labels,num_classes)
test_labels = keras.utils.to_categorical(test_labels,num_classes)



In [20]:
#Extracting the Xception model from keras
model = applications.Xception(weights='imagenet', include_top=True)
model.layers.pop()
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []

In [21]:
for layer in model.layers:
    layer.trainable = False
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [22]:
pred = Dense(10,activation='softmax')(model.output)
final_model = Model(model.input, pred)


final_model.compile(loss='categorical_crossentropy',
  optimizer='adam', metrics=['accuracy'])
gc.collect()

587

In [23]:
final_model.fit(new_train, new_train_labels, validation_split=0.04, batch_size=32, epochs=7, verbose=1)

Instructions for updating:
Use tf.cast instead.
Train on 1054 samples, validate on 44 samples
Epoch 1/7
1054/1054 [==============================] - 422s 400ms/step - loss: 2.2789 - acc: 0.4004 - val_loss: 2.2620 - val_acc: 0.4091
Epoch 2/7
1054/1054 [==============================] - 418s 396ms/step - loss: 2.2192 - acc: 0.7533 - val_loss: 2.2072 - val_acc: 0.6591
Epoch 3/7
1054/1054 [==============================] - 418s 396ms/step - loss: 2.1510 - acc: 0.8748 - val_loss: 2.1450 - val_acc: 0.7727
Epoch 4/7
1054/1054 [==============================] - 418s 396ms/step - loss: 2.0812 - acc: 0.9013 - val_loss: 2.0909 - val_acc: 0.8182
Epoch 5/7
1054/1054 [==============================] - 419s 398ms/step - loss: 2.0120 - acc: 0.9260 - val_loss: 2.0281 - val_acc: 0.8409
Epoch 6/7
1054/1054 [==============================] - 421s 399ms/step - loss: 1.9453 - acc: 0.9099 - val_loss: 1.9682 - val_acc: 0.8636
Epoch 7/7
1054/1054 [==============================] - 8506s 8s/step - loss: 1.8812 

In [24]:
final_model.evaluate(test_images, test_labels)

272/272 [==============================] - 100s 369ms/step


[1.8960128391490263, 0.9154411764705882]